In [170]:
import os 
import pandas as pd
import requests
import json
import dotenv
import lyricscraper
dotenv.load_dotenv()

True

In [171]:
# Load in client ID and secret
spotify_id = os.getenv('SpotifyID')
spotify_secret = os.getenv("SpotifySecret")

In [172]:
url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "client_credentials",
    "client_id": spotify_id,
    "client_secret": spotify_secret
}

In [173]:
# Request access token (works for one hour)
r = requests.post(url, headers=headers, data=data)
r

<Response [200]>

In [174]:
access_data = json.loads(r.text)
access_token = access_data['access_token']
token_type = access_data['token_type']

In [175]:
# Load in API keys
genius_id = os.getenv("GeniusID")
genius_secret = os.getenv("GeniusSecret")
genius_token = os.getenv("GeniusToken")
# Get user-agent 
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-agent': useragent}

In [176]:
playlist_data_full = pd.read_csv('playlist_data.csv')
playlist_data_full.head()

,Playlist_ID,Playlist_Name,Playlist_Description,Playlist_Songs,Playlist_Artists,Playlist_Song_IDs
0,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"old but gold, all the best throwback songs to ...","['Into You', 'Glad You Came', 'Dark Horse', 'W...","['Ariana Grande', 'The Wanted', 'Katy Perry', ...","['2meEiZKWkiN28gITzFwQo5', '1OXfWI3FQMdsKKC6lk..."
1,4Jb4PDWREzNnbZcOHPcZPy,COUNTRY HITS 2025 🔥 New Country Songs + Top Hits,best country music of 2025 most popular trendi...,"['I Had Some Help (Feat. Morgan Wallen)', ""Aus...","['Post Malone', 'Dasha', 'mgk', 'Dylan Marlowe...","['5IZXB5IKAD2qlvTPJYDCFB', '2uqYupMHANxnwgeiXT..."
2,2L2HwKRvUgBv1YetudaRI3,Pop 2000-2010 Bangers,"Some bangers, but no mash","['Whatcha Say', 'Airplanes (feat. Hayley Willi...","['Jason Derulo', 'B.o.B', 'Bruno Mars', 'Tinch...","['7xkQdy0cy5ymoWT7nedvLz', '1QnvpPFP4Q3FHbDchq..."
3,327k1ryJ7j1xD7gWKpdc6o,Popular Christian Music,NaN,"['Oh My Soul', 'Stars', 'Voice of Truth', 'You...","['Casting Crowns', 'Skillet', 'Casting Crowns'...","['3s0h3pyKFrS3XX6ZLBPx6s', '6VMT3SzIMbNoR5lsUs..."
4,4muHyvSwG1wP9sI4XihC5w,Instrumental Popular Songs 2025,"Instrumental Popular Songs of 2025, 2024, Upda...","['birds of a feather - piano instrumental', 'g...","['Chilled Pig', 'Chilled Pig', 'Chilled Pig', ...","['1bE4hvWlx8gtWIMVUptOXi', '1KYe04kDLflLdoJ9kN..."


In [178]:
import ast
playlist_data_full['Playlist_Songs'] = [ast.literal_eval(x) for x in playlist_data_full['Playlist_Songs']]
playlist_data_full['Playlist_Artists'] = [ast.literal_eval(x) for x in playlist_data_full['Playlist_Artists']]
playlist_data_full['Playlist_Song_IDs'] = [ast.literal_eval(x) for x in playlist_data_full['Playlist_Song_IDs']]

In [179]:
type(playlist_data_full['Playlist_Songs'][0])

list

In [180]:
genius_df = pd.read_csv("genius_url_cache.csv")

genius_url_cache = {
    (row["song"], row["artist"]): (row["genius_url"] if pd.notna(row["genius_url"]) else None)
    for _, row in genius_df.iterrows()
}
 # {(song_title, artist): genius_url}

In [182]:
# lyrics_cache = {}  # {genius_url: lyrics}
failed_song_counts = {}  # Dictionary to track missing songs

In [183]:
if failed_song_counts:
    print("\n🔹 **Most Frequently Missing Songs** 🔹")
    sorted_missing_songs = sorted(failed_song_counts.items(), key=lambda x: x[1], reverse=True)
    for (song, artist), count in sorted_missing_songs[:10]:  # Show top 10 missing songs
        print(f"{song.title()} by {artist.title()} - Missing {count} times")

In [187]:
import time
import lyricscraper as ls
import importlib
import concurrent.futures

importlib.reload(ls)

playlist_lyrics = []
playlist_data = playlist_data_full.copy()

def fetch_genius_url(song_title, artist, genius_token):
    """Helper function to fetch Genius URL with caching and failure tracking."""
    if not song_title:
        song_title = ""
    if not artist:
        artist = ""
    song_key = (song_title.lower(), artist.lower())  # Normalize case
    
    if song_key in genius_url_cache:
        return genius_url_cache[song_key]  # Use cached URL
    
    url = ls.get_genius_url(song_title, artist, genius_token)
    genius_url_cache[song_key] = url  # Store in cache

    # Track failures if the URL is None
    if url is None:
        failed_song_counts[song_key] = failed_song_counts.get(song_key, 0) + 1

    return url

def fetch_lyrics(genius_url):
    """Helper function to fetch lyrics with caching."""
    if not genius_url:
        return "Lyrics not found"
    
    if genius_url in lyrics_cache:
        return lyrics_cache[genius_url]  # Use cached lyrics
    
    lyrics = ls.scrape_lyrics(genius_url)
    lyrics_cache[genius_url] = lyrics if lyrics else "Lyrics not found"
    return lyrics_cache[genius_url]

for playlist_name, songs, artists in zip(playlist_data['Playlist_Name'], playlist_data['Playlist_Songs'], playlist_data['Playlist_Artists']):
    print(f"Grabbing Genius URLs for playlist: {playlist_name}")
    words = ['instrumental', 
             'classical', 
             'lofi',
             'lullaby version', 
             'lo fi',
             'hz',
             'french',
             'slavic',
             'russian']
    success = False
    if any(x in playlist_name.lower() for x in words):
        print(f"Skipping {playlist_name}")
        genius_urls = [None] * len(songs)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            lyrics_list = list(executor.map(fetch_lyrics, genius_urls))

        playlist_lyrics.append(lyrics_list)  # or extend(lyrics_list) if you want a flat list

        success = True
        
    while not success:
        try:
            # Use ThreadPoolExecutor to parallelize Genius URL fetching
            with concurrent.futures.ThreadPoolExecutor() as executor:
                genius_urls = list(executor.map(lambda sa: fetch_genius_url(*sa, genius_token), zip(songs, artists)))

            # Use ThreadPoolExecutor to parallelize lyrics scraping
            with concurrent.futures.ThreadPoolExecutor() as executor:
                lyrics_list = list(executor.map(fetch_lyrics, genius_urls))

            playlist_lyrics.append(lyrics_list)  # or extend(lyrics_list) if you want a flat list
            
            success = True  # If everything is successful, exit the retry loop

        except Exception as e:
            print(e)
            print(f"Error processing playlist '{playlist_name}': {e}. Retrying...")
            time.sleep(5)  # Optional delay before retrying

# Add collected lyrics to the dataframe
playlist_data = playlist_data.copy()
playlist_data["Playlist_Lyrics"] = playlist_lyrics

# Sort and print the most frequent missing songs
if failed_song_counts:
    print("\n🔹 **Most Frequently Missing Songs** 🔹")
    sorted_missing_songs = sorted(failed_song_counts.items(), key=lambda x: x[1], reverse=True)
    for (song, artist), count in sorted_missing_songs[:10]:  # Show top 10 missing songs
        print(f"{song.title()} by {artist.title()} - Missing {count} times")


Grabbing Genius URLs for playlist: Pop Hits 2000s – 2025
Grabbing Genius URLs for playlist: COUNTRY HITS 2025 🔥 New Country Songs + Top Hits
Grabbing Genius URLs for playlist: Pop 2000-2010 Bangers
Grabbing Genius URLs for playlist: Popular Christian Music
Grabbing Genius URLs for playlist: Instrumental Popular Songs 2025
Skipping Instrumental Popular Songs 2025
Grabbing Genius URLs for playlist: 2000s Pop Hit Throwback 
Grabbing Genius URLs for playlist: SONGS WE ALL KNOW😛
Grabbing Genius URLs for playlist: POP OUT AT 1 IN THE MORNING 
Grabbing Genius URLs for playlist: 2000s Pop Punk Hits
Grabbing Genius URLs for playlist: Punk Goes Pop Greatest Hits
Grabbing Genius URLs for playlist: Top Hits of 2024
Grabbing Genius URLs for playlist: 2000 R&B Hits - Top R&B 2000s Songs (R&B 00s Popular Songs)
Grabbing Genius URLs for playlist: pop girlies summer 2024!
Grabbing Genius URLs for playlist: Top Classic Rock Hits Of All Time
Grabbing Genius URLs for playlist: Rocky Training
Grabbing Geni

In [188]:
playlist_data.to_csv("Playlist_data_with_lyrics5.csv")

In [189]:
genius_df = pd.DataFrame(
    [(song, artist, url) for (song, artist), url in genius_url_cache.items()],
    columns=["song", "artist", "genius_url"]
)

genius_df.to_csv("genius_url_cache.csv", index=False)

In [190]:
lyrics_df = pd.DataFrame(lyrics_list)
lyrics_df.to_csv("lyrics_cache")

In [191]:
total_songs = sum(len(lyrics_list) for lyrics_list in playlist_data["Playlist_Lyrics"])
total_songs

184563

In [192]:
lyrics_not_found_count = sum(lyrics_list.count("Lyrics not found") for lyrics_list in playlist_data["Playlist_Lyrics"])
lyrics_not_found_count

51383

In [193]:
lyrics_not_found_count / total_songs

0.27840358034925744

In [194]:
len(genius_url_cache)

99001

In [196]:
exploded_data = playlist_data.explode(["Playlist_Artists", "Playlist_Songs", "Playlist_Song_IDs", "Playlist_Lyrics"], ignore_index=True)

In [197]:
exploded_data.groupby('Playlist_Songs').count().reset_index().sort_values(by='Playlist_ID', ascending=False).head(5)

,Playlist_Songs,Playlist_ID,Playlist_Name,Playlist_Description,Playlist_Artists,Playlist_Song_IDs,Playlist_Lyrics
14582,Daddy Issues,133,133,20,133,133,133
52725,Runaway,128,128,33,128,128,128
66332,Toxic,117,117,73,117,117,117
28122,House Of Balloons / Glass Table Girls,110,110,107,110,110,110
20467,Family Line,109,109,7,109,109,109


In [198]:
exploded_data.to_csv('Exploded_playlist_data_with_lyrics5.csv')

In [199]:
exploded_data.head(5)

,Playlist_ID,Playlist_Name,Playlist_Description,Playlist_Songs,Playlist_Artists,Playlist_Song_IDs,Playlist_Lyrics
0,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"old but gold, all the best throwback songs to ...",Into You,Ariana Grande,2meEiZKWkiN28gITzFwQo5,"[Verse 1]\nI'm so into you, I can barely breat..."
1,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"old but gold, all the best throwback songs to ...",Glad You Came,The Wanted,1OXfWI3FQMdsKKC6lkvzSx,"[Chorus: Max George]\nThe sun goes down, the s..."
2,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"old but gold, all the best throwback songs to ...",Dark Horse,Katy Perry,5jrdCoLpJSvHHorevXBATy,"[Intro: Juicy J]\nYeah, y'all know what it is\..."
3,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"old but gold, all the best throwback songs to ...",Who Knew - Edit,P!nk,4pSPMXaCjbaV3VSzZQYC7H,"[Verse 1]\nYou took my hand, you showed me how..."
4,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"old but gold, all the best throwback songs to ...",Closer,The Chainsmokers,7BKLCZ1jbUBVqRi2FVlTVw,"[Verse 1: Andrew Taggart]\nHey, I was doing ju..."
